# Hubbard Model

I'm planning on running VQE and Trotter simulation on a Hubbard Hamiltonian to better understand its structure. 

The Hubbard model is a simplification of the interactions of electrons in a solid. It can be used to explain how these interactions yield insulating, magnetic, and superconducting effects (though I don't understand any of this yet). 

In [1]:
import numpy as np
from utils import *

## Defining creation and annihilation operators 

Since we're describing electrons, the Hubbard Hamiltonian's creation and annihilation operators have the normal fermionic anticommutation relations, namely that for a fermion on site $j,k$ with spin $\sigma, \pi$, 

$$ \large \{ c_{j \sigma}, c^\dagger_{k \pi } \} = \delta_{jk} \delta_{\sigma \pi} \qquad \{ c^\dagger_{j\sigma}, c^\dagger_{k \pi} \}= \{ c_{j\sigma} c_{k\pi} \} = 0  $$

In [2]:
class Operator: 
    def __init__(self, _type, spin, qubit, dim):
        self.type = _type 
        self.spin = spin
        self.qubit = qubit
        self.dim = dim
    
#     def __mul__(self, b): 
#         return 0
        
class CreationOperator(Operator):
    def __init__(self, spin, qubit, dim):
        super(CreationOperator, self).__init__('creation', spin, qubit, dim)

class AnnihilationOperator(Operator):
    def __init__(self, spin, qubit, dim):
        super(AnnihilationOperator, self).__init__('annihilation', spin, qubit, dim)

In [3]:
c_1 = CreationOperator('down', 0, 5)
a_1 = AnnihilationOperator('up', 3, 5)

## Jordan-Wigner Transformation

In [4]:
def encode_JW(operator):
    if not issubclass(type(operator), Operator):
        raise TypeError("operator must be an Operator")
        
    result = np.array([[1.0]], dtype='complex128') 
    string_repr = ''
    # Notice the order of the tensor product: gate on 0th qubit is "leftmost" in product
    for i in range(operator.dim): 
        if i < operator.qubit: 
            result = NKron(result, Z)
            string_repr = string_repr + 'Z'
        elif i == operator.qubit and operator.type == 'creation': 
            result = NKron(result, (X -1j*Y) / 2)
            string_repr = string_repr + '-'
        elif i == operator.qubit and operator.type == 'annihilation': 
            result = NKron(result, (X +1j*Y) / 2) 
            string_repr = string_repr + '+'
        elif i > operator.qubit: 
            result = NKron(result, I) 
            string_repr = string_repr + 'I'
        else: 
            raise ValueError("Something is wrong with this code!")
    #print(string_repr)
    return result

In [5]:
x = encode_JW(c_1)

In [6]:
x.shape

(32, 32)

## Defining the Hubbard Hamiltonian

Okay, now that we have those preliminaries out of the way, what is the Hubbard Hamiltonian? What's the motivation behind it? How is it useful? 

Answer these. 

The Hubbard Hamiltonian: 
$$ \large H = -t \sum_{<j, k>, \sigma} \Big( c^\dagger_{j\sigma} c_{k\sigma} + c^\dagger_{k \sigma} c_{j \sigma} \Big) + U \sum_j n_{j \uparrow} n_{j \downarrow} - \mu \sum_j \Big(n_{j\uparrow} + n_{j\downarrow} \Big) $$

The first term is kinetic energy, a fermion moving from one site to another. The symbol $<j, k>$ implies iterating over sites that are adjacent. 

The second term is interaction energy, additional energy for a doubly-occupied site. 

The third term is chemical potential, which controls the filling. **Hm, a lot of places don't have this third term (Wikipedia even). Why?**

### 2D Hubbard Hamiltonian on a Square Lattice 

Here is a model of a 2D Hubbard Hamiltontian on a square lattice from [1811.04476](https://arxiv.org/pdf/1811.04476.pdf). We'll represent it by a 2D array: 2D to get the square structure, and in each square `[i][j]`, a dictionary of 4 elements: 'c-up', 'c-down', 'a-up', and 'a-down'. 

![](2d_hubbard.png)

In [7]:
# Make sure N <= 10!!!!!
N_X = 2
N_Y = 2
N = N_X * N_Y

In [8]:
# We use this to make iterating over neighbors easier 
# NOTE: in HubbardHamiltonian2D, we generate the JW transformation instead of the operator class
square_lattice = [[ {'i': i, 'j': j, 
                     'c-up': CreationOperator('UP', N_Y * i + j, N), 
                     'c-down': CreationOperator('DOWN', N_Y * i + j, N), 
                     'a-up': AnnihilationOperator('UP', N_Y * i + j, N), 
                     'a-down': AnnihilationOperator('DOWN', N_Y * i + j, N)
                    } for j in range(N_X)] for i in range(N_Y)]

In [9]:
square_lattice[0][1]

{'i': 0,
 'j': 1,
 'c-up': <__main__.CreationOperator at 0x7fde1750d748>,
 'c-down': <__main__.CreationOperator at 0x7fde1750d7b8>,
 'a-up': <__main__.AnnihilationOperator at 0x7fde1750d7f0>,
 'a-down': <__main__.AnnihilationOperator at 0x7fde1750d828>}

In [10]:
class HubbardHamiltonian2D: 
    def __init__(self, t, U, mu, N_X, N_Y):
        self.t = t 
        self.U = U 
        self.mu = mu 
        self.N_X = N_X 
        self.N_Y = N_Y
        self.N = N_X * N_Y 
        self.square_lattice = self._gen_square_lattice()
        self.hamiltonian = self._gen_hamiltonian()
        
    def _gen_square_lattice(self):
        # We use this to make iterating over neighbors easier 
        return [[ {'i': i, 'j': j, 
                   'c-up': encode_JW(CreationOperator('UP', self.N_Y * i + j, self.N)), 
                   'c-down': encode_JW(CreationOperator('DOWN', self.N_Y * i + j, self.N)), 
                   'a-up': encode_JW(AnnihilationOperator('UP', self.N_Y * i + j, self.N)), 
                   'a-down': encode_JW(AnnihilationOperator('DOWN', self.N_Y * i + j, self.N))
                  } for j in range(self.N_X)] for i in range(self.N_Y)]
    
    def _gen_hamiltonian(self): 
        return (-self.t * (self._gen_hor_even_sum() + self._gen_hor_odd_sum() + 
                           self._gen_ver_even_sum() + self._gen_ver_odd_sum()) + 
                self.U * self._gen_interaction_energy())
    
    # I'm calculating the Hamiltonian like this because we'll be separating these terms later for the ansatz
    def _gen_hor_even_sum(self): 
        total = np.eye(2**self.N, dtype='complex128')
        for i in range(self.N_Y): 
            for j in range(self.N_X): 
                if j % 2 == 0 and j + 1 < self.N_X: 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i][j+1]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
        return total 
    
    def _gen_hor_odd_sum(self): 
        total = np.eye(2**self.N, dtype='complex128')
        for i in range(self.N_Y): 
            for j in range(self.N_X): 
                if j % 2 == 1 and j + 1 < self.N_X: 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i][j+1]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
        return total 
    
    def _gen_ver_even_sum(self):
        total = np.eye(2**self.N, dtype='complex128')
        for i in range(self.N_Y): 
            if i % 2 == 0 and i + 1 < self.N_Y: 
                for j in range(self.N_X): 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i+1][j]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
        return total 
    
    def _gen_ver_odd_sum(self): 
        total = np.eye(2**self.N, dtype='complex128')
        for i in range(self.N_Y): 
            if i % 2 == 1 and i + 1 < self.N_Y: 
                for j in range(self.N_X): 
                    site = self.square_lattice[i][j]
                    next_site = self.square_lattice[i+1][j]
                    total += NDot(site['c-up'], next_site['a-up'])
                    total += NDot(site['c-down'], next_site['a-down'])
                    total += NDot(next_site['c-up'], site['a-up'])
                    total += NDot(next_site['c-down'], site['a-down'])
        return total 
    
    def _gen_interaction_energy(self):
        total = np.eye(2**self.N, dtype='complex128')
        for i in range(self.N_Y): 
            for j in range(self.N_X): 
                site = self.square_lattice[i][j] 
                total += NDot(site['c-up'], site['a-up'], site['c-down'], site['a-down'])
        return total

In [11]:
x = HubbardHamiltonian2D(3, 4, 5, N_X, N_Y)